## Final Assignment


Before working on this assignment please read these instructions fully. Use blackboard to submit a link to your repository. 

On blackboard your find the assessment criteria. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find at least two datasets on the web which are related, and that you build an application that visualize these datasets to answer a research question with the broad topic of **health** or **agriculture** in the **region where you were born**. The region can be a city, town or a provence.  

The research question should be a question with a causual nature. For instance questions like: How does independent variable X influence the dependent variable of Y?

The code should be programmed efficiently. Also identify the most critical part and write software test for this part. Take into account the performance of the dataprocessing

### About the data

You can merge these datasets with data from different regions if you like. For instance, you might want to compare the health effect of earhtquacks in Groningen versus Los Angelos USA. 

You are welcome to choose datasets at your discretion, but keep in mind they will be shared with others, so choose appropriate datasets. You are welcome to use datasets of your own as well, but minimual two datasets should be coming from the web and or API's. 

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations in your visualization. 

### Instructions:

Define a research question, select data and code your data acquisition, data processing, data analysis and visualization. Write code to test most critical parts. Use a repository with a commit strategy and write a readme file. 

Write a small document with the following:
- State the region and the domain category that your data sets are about 
- State the research question 
- Justify the chosen data storage and processing approach
- Justify the chosen analysis approach
- Justify the chosen data visualization approach

Upload your document and the link of your repository to black board

## loading in the datafiles and taking a first look

In [19]:
import json
import pandas as pd
pd.options.mode.chained_assignment = None

def load_in(path_sewage, path_cases):
    sewage = pd.read_json(path_sewage, orient= "levels")
    cases = pd.read_json(path_cases, orient= "levels")
    sewage.head()
    cases.head()

    return sewage, cases

df_sewage, df_cases = load_in("Data/COVID-19_rioolwaterdata.json", 
                              "Data/COVID-19_aantallen_gemeente_cumulatief.json")

## Filtering
For the filtering of the data the columns that are uneccesary for this research will be removed.
Next there will be a check for missing data
For the sewage data there will be looked at which measure points have been used 
throughout and the ones that have been added later.The measure points added later will be removed.

Both datasets will be filtered on date. For an acurate date the sewage dataset is used. This is done
by checking when they started wit measuring the RNA flow per 10.000 inhibitants instead of RNA per ml.
This is done because it nearly impossible to calculate the RNA flow from the RNA per ml.
The end date is the same as the end date of the last entry in the sewage dataset.

Because of the many different measurepoints the RNA flow per 10.000 will be summed per province.
This will also be done for the total amount of corona cases.



#### sewage dataset

In [29]:
# deleting uneccesary columns:

#df_sewage = df_sewage.drop(["RWZI_AWZI_code", "RWZI_AWZI_name", "X_coordinate", "Y_coordinate",
#                           "Postal_code", "Security_region_code", "RNA_per_ml"], axis = 1)

# check unique values in the columns percentage_in_security_regrion and Representative_measurement to check 
# if there are any differences:

print(df_sewage['Percentage_in_security_region'].nunique())
print(df_sewage['Representative_measurement'].nunique())
# with this we can remove the representative column because all values are TRUE
#df_sewage = df_sewage.drop(['Representative_measurement'], axis = 1)



119
1
26


In [62]:
df_sewage


,Date_measurement,Security_region_name,Percentage_in_security_region,RNA_flow_per_100000,Province
0,2020-03-30,Midden- en West-Brabant,1,,Noord Brabant
1,2020-03-31,Drenthe,1,,Drenthe
2,2020-03-31,Groningen,1,,Groningen
3,2020-03-31,Friesland,1,,Friesland
4,2020-03-31,Friesland,1,,Friesland
...,...,...,...,...,...
10515,2021-01-27,Gelderland-Midden,1,12320280273098,Gelderland
10516,2021-01-27,Limburg-Noord,1,17615099163688,Limburg
10517,2021-01-27,Midden- en West-Brabant,1,20535140783432,Noord Brabant
10518,2021-01-27,Gelderland-Midden,1,7748971717648,Gelderland


In [59]:
import numpy as np
# check the number of unique values in the Security region name.
df_sewage['Security_region_name'].nunique()

# from this we can see that there are 26 regions measured. However there are only twelve provinces.
# We will check per region name in which province they belong and add them in an additional column.

df_sewage.Security_region_name.unique()
df_sewage["Province"] = np.nan

df_sewage.loc[df_sewage.Security_region_name == "Midden- en West-Brabant", 'Province'] = "Noord Brabant"
df_sewage.loc[df_sewage.Security_region_name == "Brabant-Noord", 'Province'] = "Noord Brabant"
df_sewage.loc[df_sewage.Security_region_name == "Brabant-Zuidoost", 'Province'] = "Noord Brabant"

df_sewage.loc[df_sewage.Security_region_name == "IJsselland", 'Province'] = "Overijssel"
df_sewage.loc[df_sewage.Security_region_name == "Ijsselland", 'Province'] = "Overijssel"
df_sewage.loc[df_sewage.Security_region_name == "Twente", 'Province'] = "Overijssel"

df_sewage.loc[df_sewage.Security_region_name == "Gelderland-Midden", 'Province'] = "Gelderland"
df_sewage.loc[df_sewage.Security_region_name == "Gelderland-Zuid", 'Province'] = "Gelderland"
df_sewage.loc[df_sewage.Security_region_name == "Noord- en Oost-Gelderland", 'Province'] = "Gelderland"

df_sewage.loc[df_sewage.Security_region_name == "Kennemerland", 'Province'] = "Noord Holland"
df_sewage.loc[df_sewage.Security_region_name == "Noord-Holland-Noord", 'Province'] = "Noord Holland"
df_sewage.loc[df_sewage.Security_region_name == "Zaanstreek-Waterland", 'Province'] = "Noord Holland"
df_sewage.loc[df_sewage.Security_region_name == "Amsterdam-Amstelland", 'Province'] = "Noord Holland"
df_sewage.loc[df_sewage.Security_region_name == "Gooi en Vechtstreek", 'Province'] = "Noord Holland"

df_sewage.loc[df_sewage.Security_region_name == "Hollands-Midden", 'Province'] = "Zuid Holland"
df_sewage.loc[df_sewage.Security_region_name == "Haaglanden", 'Province'] = "Zuid Holland"
df_sewage.loc[df_sewage.Security_region_name == "Rotterdam-Rijnmond", 'Province'] = "Zuid Holland"
df_sewage.loc[df_sewage.Security_region_name == "Zuid-Holland-Zuid", 'Province'] = "Zuid Holland"

df_sewage.loc[df_sewage.Security_region_name == "Limburg-Noord", 'Province'] = "Limburg"
df_sewage.loc[df_sewage.Security_region_name == "Limburg-Zuid", 'Province'] = "Limburg"

df_sewage.loc[df_sewage.Security_region_name == "Drenthe", 'Province'] = "Drenthe"
df_sewage.loc[df_sewage.Security_region_name == "Groningen", 'Province'] = "Groningen"
df_sewage.loc[df_sewage.Security_region_name == "Friesland", 'Province'] = "Friesland"
df_sewage.loc[df_sewage.Security_region_name == "Utrecht", 'Province'] = "Utrecht"
df_sewage.loc[df_sewage.Security_region_name == "Flevoland", 'Province'] = "Flevoland"
df_sewage.loc[df_sewage.Security_region_name == "Zeeland", 'Province'] = "Zeeland"

